<a href="https://colab.research.google.com/github/RuthlessActuary2023/CAS-RPM-2025-Python-Workshop/blob/main/2025_CAS_RPM_Python_Lesson_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lesson 5: Visualizing Data - Advanced Folium Map

## Import packages & data

In [ ]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns",25)
pd.set_option("display.max_rows",250)
pd.options.display.float_format = '{:,.2f}'.format

from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 90%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

import matplotlib.pyplot as plt

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from urllib.request import urlopen
import json

In [ ]:
# Code to clone the github repo into colab
!pip install gitpython
import git
import os

repo_url = 'https://github.com/RuthlessActuary2023/CAS-RPM-2025-Python-Workshop.git'
repo_dir = '/content/your-repo'

# Clone if it doesn’t already exist
if not os.path.exists(repo_dir):
    git.Repo.clone_from(repo_url, repo_dir)

In [ ]:
repo_dir = '/content/your-repo'
data5 = pd.read_feather(repo_dir + '/data5.ftr')

In [ ]:
[i for i in data5.columns]

In [ ]:
### Function to make useful Collision stats
facts = ['ee_col','incloss_col','cc_col','ep_col']

def stats_by_zip(data):
    z = data.groupby(['zip']).agg({f: 'sum' for f in facts}).reset_index()

    z['incloss_col'] = np.where(z['incloss_col'] < 0, 0, z['incloss_col'])

    z['Exposure %'] = z['ee_col']/z['ee_col'].sum()
    z['Frequency'] = round(z['cc_col']/z['ee_col'],3)
    z['Severity'] = round(z['incloss_col']/z['cc_col'],0)
    z['Pure Premium'] = round(z['incloss_col']/z['ee_col'],0)
    z['Loss Ratio'] = round(z['incloss_col']/z['ep_col'],0)

    return z[(['zip','Exposure %','Frequency','Severity','Pure Premium','Loss Ratio'])]

In [ ]:
stats_by_zip(data5)

## Folium Map + modifications

In [ ]:
path = 'C:/Users/marcusdeckert/Box/marcusdeckert/Presentation Work/CARFAX Academy/Lesson 5 stuff/'

import folium

def loss_ratio_folium_map(data):
    zip_data = stats_by_zip(data)
    zip_data['zip'] = zip_data['zip'].astype('str') #skipped accounting for zip codes with leading 0's because we are doing Ohio

    with open(path + 'oh_ohio_zip_codes_geo.min.json') as f:
        zips_json = json.load(f)

    m = folium.Map(location = [40, -83], zoom_start = 8, stroke = False, prefer_canvas=True)

    choro = folium.Choropleth(
            geo_data = zips_json,
            data = zip_data,
            columns = ['zip','Loss Ratio'],
            key_on = 'feature.properties.ZCTA5CE10',
            fill_color = 'YlOrRd',
            nan_fill_color = 'gray',
            fill_opacity = 0.5,
            line_opacity = 0.0,
            legend_name = 'Loss Ratio',
        )

    choro.add_to(m)

    # Title
    title = 'Loss Ratio by Zip Code'
    title_html = '''
                 <h3 align="center" style="font-size:16px"><b>{}</b></h3>
                 '''.format(title)
    m.get_root().html.add_child(folium.Element(title_html))

    return m

In [ ]:
loss_ratio_folium_map(data5)

In [ ]:
# limit to p5 and p95
def loss_ratio_folium_map2(data):
    zip_data = stats_by_zip(data)
    zip_data['zip'] = zip_data['zip'].astype('str')

    ### SECTION ADDED
    # limit the range of values to p5 and p95

    min_val = zip_data['Loss Ratio'].quantile(q = .05)
    max_val = zip_data['Loss Ratio'].quantile(q = .95)

    zip_data['Loss Ratio Display'] = np.where(
        zip_data['Loss Ratio'] < min_val, min_val,
        np.where(
            zip_data['Loss Ratio'] > max_val, max_val, zip_data['Loss Ratio']
        )
    )

    ### END SECTION ADDED

    with open(path + 'oh_ohio_zip_codes_geo.min.json') as f:
        zips_json = json.load(f)

    m = folium.Map(location = [40, -83], zoom_start = 8, stroke = False, prefer_canvas=True)

    choro = folium.Choropleth(
            geo_data = zips_json,
            data = zip_data,
            columns = ['zip','Loss Ratio Display'], ### CHANGED
            key_on = 'feature.properties.ZCTA5CE10',
            fill_color = 'YlOrRd',
            nan_fill_color = 'gray',
            fill_opacity = 0.5,
            line_opacity = 0.0,
            legend_name = 'Loss Ratio',
        )

    choro.add_to(m)

    # Title
    title = 'Loss Ratio by Zip Code; p5 to p95'
    title_html = '''
                 <h3 align="center" style="font-size:16px"><b>{}</b></h3>
                 '''.format(title)
    m.get_root().html.add_child(folium.Element(title_html))

    return m

In [ ]:
loss_ratio_folium_map2(data5)

In [ ]:
# change to [0,1]
def loss_ratio_folium_map3(data):
    zip_data = stats_by_zip(data)
    zip_data['zip'] = zip_data['zip'].astype('str')

    min_val = 0 ### CHANGED
    max_val = 1 ### CHANGED

    zip_data['Loss Ratio Display'] = np.where(
        zip_data['Loss Ratio'] < min_val, min_val,
        np.where(
            zip_data['Loss Ratio'] > max_val, max_val, zip_data['Loss Ratio']
        )
    )

    with open(path + 'oh_ohio_zip_codes_geo.min.json') as f:
        zips_json = json.load(f)

    m = folium.Map(location = [40, -83], zoom_start = 8, stroke = False, prefer_canvas=True)

    choro = folium.Choropleth(
            geo_data = zips_json,
            data = zip_data,
            columns = ['zip','Loss Ratio Display'],
            key_on = 'feature.properties.ZCTA5CE10',
            fill_color = 'YlOrRd',
            nan_fill_color = 'gray',
            fill_opacity = 0.5,
            line_opacity = 0.0,
            legend_name = 'Loss Ratio',
        )

    choro.add_to(m)

    # Title
    title = 'Loss Ratio by Zip Code; Limit 100%'
    title_html = '''
                 <h3 align="center" style="font-size:16px"><b>{}</b></h3>
                 '''.format(title)
    m.get_root().html.add_child(folium.Element(title_html))

    return m

In [ ]:
loss_ratio_folium_map3(data5)

## Add Hoverinfo

In [ ]:
### First, let's look at the geojson again

with open(path + 'oh_ohio_zip_codes_geo.min.json') as f:
    zips_json = json.load(f)

zips_json['features'][0]

In [ ]:
### We need the geojson to contain the needed information so it can be displayed as a "child"

# We want to have the pop up show Zip, Exposure %,Frequency,Severity,Pure Premium,Loss Ratio

# Let's begin by simply adding an element called 'Zip' to a copy of the geojson

geo_data = zips_json.copy()
for i in range(0,len(geo_data['features']) - 1): # consider refactoring these 2 lines to directly call each element
    geo_data['features'][i]['properties']['Zip'] = geo_data['features'][i]['properties']['ZCTA5CE10'] # take this straight from the JSON

geo_data['features'][0]

In [ ]:
### We need the other statistics to show in hoverinfo from the dataframe
# pick this first zip and grab the values

data = stats_by_zip(data5)

data.loc[data['zip'] == 45830]

In [ ]:
### Get just the Exposure %

data.loc[data['zip'] == 45830]['Exposure %'].sum()

In [ ]:
# format it nicely

'{:.1%}'.format(data.loc[data['zip'] == 45830]['Exposure %'].sum())

In [ ]:
### Add the rest of the information now
data = stats_by_zip(data5)

geo_data = zips_json.copy()
for i in range(0,len(geo_data['features']) - 1): # if you redid the one above do this one, too
    geo_data['features'][i]['properties']['Zip'] = geo_data['features'][i]['properties']['ZCTA5CE10'] # take this straight from the JSON
    geo_data['features'][i]['properties']['Exposure %'] = 'Exposure %: ' + '{:.1%}'.format(data.loc[data['zip'] == geo_data['features'][i]['properties']['Zip']]['Exposure %'].sum())
    geo_data['features'][i]['properties']['Frequency'] = 'Frequency: ' + '{:.1%}'.format(data.loc[data['zip'] == geo_data['features'][i]['properties']['Zip']]['Frequency'].sum())
    geo_data['features'][i]['properties']['Severity'] = 'Severity: ' + '{:,}'.format(data.loc[data['zip'] == geo_data['features'][i]['properties']['Zip']]['Severity'].sum())
    geo_data['features'][i]['properties']['Pure Premium'] = 'Pure Premium: ' + '{:,}'.format(data.loc[data['zip'] == geo_data['features'][i]['properties']['Zip']]['Pure Premium'].sum())
    geo_data['features'][i]['properties']['Loss Ratio'] = 'Loss Ratio: ' + '{:.1%}'.format(data.loc[data['zip'] == geo_data['features'][i]['properties']['Zip']]['Loss Ratio'].sum())

geo_data['features'][0]

In [ ]:
### Add this into the map function and add the child

In [ ]:
def loss_ratio_folium_map4(data):
    data = stats_by_zip(data) ### Changed
    data['zip'] = data['zip'].astype('str') ### Changed

    min_val = 0
    max_val = 1

    ### Changed
    data['Loss Ratio Display'] = np.where(
        data['Loss Ratio'] < min_val, min_val,
        np.where(
            data['Loss Ratio'] > max_val, max_val, data['Loss Ratio']
        )
    )

    with open(path + 'oh_ohio_zip_codes_geo.min.json') as f:
        zips_json = json.load(f)

    ### Added
    geo_data = zips_json.copy()
    for i in range(0,len(geo_data['features']) - 1):
        geo_data['features'][i]['properties']['Zip'] = geo_data['features'][i]['properties']['ZCTA5CE10'] # take this straight from the JSON
        geo_data['features'][i]['properties']['Exposure %'] = 'Exposure %: ' + '{:.1%}'.format(data.loc[data['zip'] == geo_data['features'][i]['properties']['Zip']]['Exposure %'].sum())
        geo_data['features'][i]['properties']['Frequency'] = 'Frequency: ' + '{:.1%}'.format(data.loc[data['zip'] == geo_data['features'][i]['properties']['Zip']]['Frequency'].sum())
        geo_data['features'][i]['properties']['Severity'] = 'Severity: ' + '{:,}'.format(data.loc[data['zip'] == geo_data['features'][i]['properties']['Zip']]['Severity'].sum())
        geo_data['features'][i]['properties']['Pure Premium'] = 'Pure Premium: ' + '{:,}'.format(data.loc[data['zip'] == geo_data['features'][i]['properties']['Zip']]['Pure Premium'].sum())
        geo_data['features'][i]['properties']['Loss Ratio'] = 'Loss Ratio: ' + '{:.1%}'.format(data.loc[data['zip'] == geo_data['features'][i]['properties']['Zip']]['Loss Ratio'].sum())
    ### END Added

    m = folium.Map(location = [40, -83], zoom_start = 8, stroke = False, prefer_canvas=True)

    choro = folium.Choropleth(
            geo_data = geo_data, ### Changed
            data = data, ### Changed
            columns = ['zip','Loss Ratio Display'],
            key_on = 'feature.properties.ZCTA5CE10',
            fill_color = 'YlOrRd',
            nan_fill_color = 'gray',
            fill_opacity = 0.5,
            line_opacity = 0.0,
            legend_name = 'Loss Ratio',
        )

    ### Added
    for key in choro._children:
        if key.startswith('color_map'):
            del(choro._children[key])
    ### END Added

    choro.add_to(m)

    ### Added
    folium.LayerControl().add_to(m)
    choro.geojson.add_child(
        folium.features.GeoJsonTooltip(['Zip','Exposure %','Frequency','Severity','Pure Premium','Loss Ratio'], labels=False)
    )
    ### END Added

    # Title
    title = 'Loss Ratio by Zip Code; Limit 100%'
    title_html = '''
                 <h3 align="center" style="font-size:16px"><b>{}</b></h3>
                 '''.format(title)
    m.get_root().html.add_child(folium.Element(title_html))

    return m

In [ ]:
loss_ratio_folium_map4(data5)

## Accessing more information from the geojson

In [ ]:
with open(path + 'oh_ohio_zip_codes_geo.min.json') as f:
    zips_json = json.load(f)

geo_data = zips_json.copy()

x = geo_data['features'][0]

In [ ]:
x2 = pd.DataFrame(x).T

In [ ]:
x2

In [ ]:
zip_df = stats_by_zip(data5)
zip_df['zip'] = zip_df['zip'].astype('str')

x3 = x2.merge(zip_df, left_on = 'ZCTA5CE10', right_on = 'zip', how = 'left')

x3